<a href="https://colab.research.google.com/github/sjdoescoding/CE156-Assignment/blob/main/Exercise%205/%20CE156_Exercise_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5
1. Write a program that will open a file, read the firs tline and create a 2-dimensional NumPy array, where number of rows is the number of students and each row has 4 elements (marks). Initialise this array using a call to the array function with an argument ([[0,0,0,0]])*n, where n is the number of students.

2. The program should then read the remaining lines, line-by-line and use the input values to store in a row of the array the registration number, the exam mark, the average coursework mark and overall mark (calculated using the exam mark, the average coursework mark and the weighting.) You may assume that all lines will contain the correct number of marks and the number of students will match the number specified on the first line.

3. Having generated the array, the program should then create another 1-dimensional array with n elements, but with a structured data type with 4 integers and one string as the components. (use a list containing multiple copies of a structure with default values (e.g. zeroes and an empty string) as the argument to array)

4. From the data in each row of the first array the program should generate a structure containing the student's registration number, exam mark, coursework mark and total mark, all rounded to the nearest integer, and a grade string to be calculated using the rules below, and store this in the corresponding element in the second array. (You can round a real number n to the nearest integer using round(n)

5. The program should then produce a version of the second array sorted by overall mark and output this array to a file (using a single call to the print function of the form print(array2, file=f)

6. It should finally output to the screen/console the number of students who have first, second and third-class marks, the number of students who have failed and the registration of the students who have first-class marks.

Rules for calculating grades
Any student with a rounded mark of less than 30 for either the coursework average or the exam has 
failed, irrespective of the overall module mark. For all other students a rounded overall mark of 70 or 
more has a first-class grade, a rounded mark between 50 and 69 (inclusive) has a second-class grade, a 
rounded mark between 40 and 49 has a third-class grade, and a rounded mark below 40 fails

In [ ]:
## Import libraries
import numpy as np
import pandas as pd


# Open file for reading
with open('ex5data.txt', 'r') as file:
    file.readline() # Reads the first line of the file
    arr = [(line.strip()).split(" ") for line in file] # Splits the files by line into lists
    arr2 = np.array(arr) # Stores lists from arr to an array arr2
  
    
    # Coursework = 40%, Exam = 60%
    # i.e. 37 * 0.4 = 14.8, 93 * 0.6 = 55.8 
    # overall mark = 14.8 + 55.8 = 70.6666
    ## Finding the average coursework marks
    arr3 = arr2[:,2:].astype(np.float) # Converts the final 3 columns (coursework marks) to float value
    mean_coursework = arr3.mean(axis=1) # calculates the means between the coursework columns
    mean_coursework = np.reshape(mean_coursework, (-1, 1)) # reshapes the coursework marks into a single 2d array column
    weighted_coursework = mean_coursework * 0.4 # Multiply coursework by 40% to add weighting
    rounded_weighted_coursework = np.rint(weighted_coursework) # Converts float value into int, rounded to nearest integer, may just keep it as float
    


    ## Converting Exam marks to float
    exam = arr2[:,1:2:].astype(np.float) # Slices exam marks and converts it to float
    exam = np.around(exam, decimals=0, out=None)
    weighted_exam = exam * 0.6 # Multiply exam by 60% to add weighting 
    rounded_weighted_exam = np.rint(weighted_exam)


    # Calculate overall mark with weightings
    overall_mark = rounded_weighted_exam + rounded_weighted_coursework


    # Creates new array
    new_arr = np.concatenate((arr2[:,0:1], exam, rounded_weighted_coursework, overall_mark), axis =1)



# Create Pandas df and input new_arr array, column headers ['Registration', Exam Mark', 'Coursework Average', 'Overall Mark']
df = pd.DataFrame(new_arr, columns=['Registration', 'Exam Mark', 'Coursework Average','Overall Mark'])
# Converts all values in dataframe to float, then converts all values to integer
df = df.astype(float).astype(int)


## Function that takes the argument row
## if/elif statements checks if row in Overall Mark Column is equal to ranges given 
def match_grades(row):
   if row['Overall Mark'] >= 70: # if overall mark column row is equal or greater than 70, return 'First Class'
      return 'First Class'
   elif row['Overall Mark'] < 70 and row['Overall Mark'] >= 50: # if overall mark column row is less than 70 and overall mark column is equal or greater than 50, return 'Second Class'
      return 'Second Class'
   elif row['Overall Mark'] < 50 and row['Overall Mark'] >= 40: # if overall mark column row is less than 50 and overall mark is equal or greater than 40, return 'Third class'
      return 'Third Class'
   elif row['Overall Mark'] < 40: # if overall mark column row is less than 40, return 'fail'
      return 'Fail'

## lambda function runs match_grades(row) method, and df.apply applies the return strings to the newly created df['Grades'] column 
df['Grades'] = df.apply(lambda row: match_grades(row), axis=1)

# Sorts df from highest 'Overall Mark' to lowest
df = df.sort_values('Overall Mark', ascending = False)

# Prints DataFrame, with a new line for output clarity
print(df, '\n') 


# Converts df to numpy array, writes array to a file f
array2 = df.to_numpy()
with open("output.txt", "a+") as f:
  print(array2, file=f)

# Outputs number of students with Second class, Thirs Class, First Class and Fail Marks
print(df['Grades'].value_counts())

# Outputting Registration numbers of students with First class marks
empty_lst = [] # Initialize list to append Registration column
for index, row in df.iterrows(): # Iterates rows of dataframe columns
    if row["Grades"] == 'First Class': # Checks if 'First Class' is in ['Grades']
      empty_lst.append(row['Registration']) # Append Registration number to list if it matches 'First Class'

      
print('\n' 'The registration numbers of students with First Class Marks:', empty_lst) # Registration number output


    Registration  Exam Mark  Coursework Average  Overall Mark       Grades
87       2147897         99                  26            85  First Class
71       2076307         99                  25            84  First Class
93       2174630         99                  19            78  First Class
72       2082765         99                  19            78  First Class
63       2047582         99                  19            78  First Class
..           ...        ...                 ...           ...          ...
26       1897777         33                  13            33         Fail
75       2084051         10                  26            32         Fail
51       2005841          2                  26            27         Fail
98       2197297          0                  19            19         Fail
79       2124944          4                  17            19         Fail

[100 rows x 5 columns] 

Second Class    52
Third Class     19
First Class     15
Fail            1